# Python script to plot data from 01EMMA_CYL

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from netCDF4 import Dataset
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
import netCDF4



## Choose a netyCDF file to read 

In [5]:
file_name = 'cc_etat013.nc'

url_base = 'http://servdap.legi.grenoble-inp.fr:80/opendap/coriolis/01EMMA_CYL/cc/'
url = url_base + file_name

rootgrp = Dataset(url)

## plotting function

In [6]:
def g(variable, setcolorbar, varmax, varmin, vector):
    fig_xsize = 15
    fig_ysize = 10
    font_size = 25
    markersize = 10
    fig = plt.figure(num=0, figsize=(fig_xsize, fig_ysize), dpi=60,
                    facecolor='w', edgecolor='w')
    xaxis_name = 'i' #'x' " (" + rootgrp.variables['xbrut1'].units + ")"
    yaxis_name = 'j' #'y' + " (" + rootgrp.variables['ybrut1'].units + ")"
    plt.rcParams.update({'font.size': font_size})
    plt.rcParams['lines.markersize'] = 10
    XX = np.linspace(0, 49, 50)
    X = np.empty([50, 50])
    Y = np.empty([50, 50])
    X[0:50, :] = XX
    Y = X.T
    contourcolor = plt.cm.coolwarm
    if vector:
        norm = np.sqrt(rootgrp.variables['u'][0, 0, :, :]**2 + rootgrp.variables['v'][0, 0, :, :]**2)
        if setcolorbar:
            levels = np.linspace(varmin, varmax, 100)
            mycontour = plt.contourf(norm, levels, cmap=contourcolor)
        else:
            mycontour = plt.contourf(norm, 100, cmap=contourcolor)
        cbar = plt.colorbar(mycontour)
        plt.quiver(X, Y, rootgrp.variables['u'][0, 0, :, :], rootgrp.variables['v'][0, 0, :, :])
        cbar.ax.set_ylabel('Velocity (m/s)')
    else:
        varplot = rootgrp.variables[variable][0, 0, :, :]
        if setcolorbar:
            levels = np.linspace(varmin, varmax, 100)
            mycontour = plt.contourf(varplot, levels, cmap=contourcolor)
        else:
            mycontour = plt.contourf(varplot, 100, cmap=contourcolor)
        cbar = plt.colorbar(mycontour)
        cbar.ax.set_ylabel(rootgrp.variables[variable].name + " (" + rootgrp.variables[variable].units + ")")
    plt.gca().set_aspect('equal')
    #plt.ylim(ymin, ymax)
    #plt.xlim(xmin, xmax)
    plt.xlabel(xaxis_name)
    plt.ylabel(yaxis_name)
    plt.show()

In [7]:
var = []
for i in rootgrp.variables.keys():
    if np.shape(rootgrp.variables[i]) == (1, 2, 50, 50) and i != 'hauteur_de_couche':
        var.append(i)

interact(g, variable=var, setcolorbar= False, varmin=widget.FloatText(value=-0.006,step=0.0001,description='value min:',disabled=False), varmax=widget.FloatText(value=0.006,step=0.0001,description='value max:',disabled=False),vector=widget.Checkbox(value=False, description='Display vectors',disabled=False));

interactive(children=(Dropdown(description='variable', options=('u', 'v'), value='u'), Checkbox(value=False, description='setcolorbar'), FloatText(value=0.006, description='value max:', step=0.0001), FloatText(value=-0.006, description='value min:', step=0.0001), Checkbox(value=False, description='Display vectors'), Output()), _dom_classes=('widget-interact',))